
# OpenProBono RAG Evaluation

This notebook is based on [RAG Evaluation](https://huggingface.co/learn/cookbook/en/rag_evaluation) by [Aymeric Roucher](https://huggingface.co/m-ric).

Any section inside block quotes is a direct quote. The general structure is copied and ideas are paraphrased. Prompts are adjusted to fit OpenProBono's use case. Code has been added and modified.

>This notebook demonstrates how you can evaluate your RAG (Retrieval Augmented Generation), by building a synthetic evaluation dataset and using LLM-as-a-judge to compute the accuracy of your system.
>
>For an introduction to RAG, you can check [this other cookbook](https://huggingface.co/learn/cookbook/en/rag_zephyr_langchain)!
>
>RAG systems are complex: here a RAG diagram, where we noted in blue all possibilities for system enhancement:
>
><img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" alt="RAG workflow: Knowledge Base, Embedding Model, LLM, LLM Prompt, and more" height="700"/>
>
>Implementing any of these improvements can bring a huge performance boost; but changing anything is useless if you cannot monitor the impact of your changes on the system’s performance! So let’s see how to evaluate our RAG system.
>

### Evaluating RAG performance

>Since there are so many moving parts to tune with a big impact on performance, benchmarking the RAG system is crucial.
>
>For our evaluation pipeline, we will need:
>
>1. An evaluation dataset with question - answer couples (QA couples)
>2. An evaluator to compute the accuracy of our system on the above evaluation dataset.
>
>➡️ It turns out, we can use LLMs to help us all along the way!
>
>1. The evaluation dataset will be synthetically generated by an LLM 🤖, and questions will be filtered out by other LLMs 🤖
>2. An [LLM-as-a-judge](https://huggingface.co/papers/2306.05685) agent 🤖 will then perform the evaluation on this synthetic dataset.
>
>**Let’s dig into it and start building our evaluation pipeline!**

### 0: Install and import dependencies

In [45]:
%pip install -q tqdm openai pandas langchain unstructured pymilvus

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
import os
from pathlib import Path

import openai
import pandas as pd
import pymilvus
from tqdm.auto import tqdm

import chat_models
import encoders
import milvusdb

### 1: Build a synthetic dataset for evaluation

>We first build a synthetic dataset of questions and associated contexts. The method is to get elements from our knowledge base, and ask an LLM to generate questions based on these documents.
>
>Then we setup other LLM agents to act as quality filters for the generated QA couples: each of them will act as the filter for a specific flaw.

#### 1.1: Load sources
 
We have a list of sources we use to generate questions about one source at a time using a generator function. The sources can be files or URLs.

Our current sources are the NC General Statutes. We can access them by chapter or by statute.

In [ ]:
from unstructured.partition.auto import partition

# for loading by statute
root_dir = "chapter_urls/"
chapter_names = sorted(os.listdir(root_dir))
# for loading by chapter
with Path("urls").open() as f:
    chapter_pdf_urls = [line.strip() for line in f.readlines()]

def load_statute_urls(chapter_name: str):
    with Path(root_dir + chapter_name).open() as f:
        return [line.strip() for line in f.readlines()]

def generate_statute_elements():
    for chapter in chapter_names:
        statute_urls = load_statute_urls(chapter)
        for statute_url in statute_urls:
            elements = partition(statute_url)
            yield statute_url, elements

def resume_statute_elements(chapter_name: str, statute_url: str):
    resume_chapter = next(iter([chapter for chapter in chapter_names if chapter == chapter_name]), None)
    resume_chapter_idx = chapter_names.index(resume_chapter) if resume_chapter else 0
    for i in range(resume_chapter_idx, len(chapter_names)):
        statute_urls = load_statute_urls(chapter_names[i])
        resume_statute = next(iter([statute for statute in statute_urls if statute == statute_url]), None)
        resume_statute_idx = statute_urls.index(resume_statute) if resume_statute else 0
        for j in range(resume_statute_idx, len(statute_urls)):
            elements = partition(url=statute_urls[j], content_type="application/pdf")
            yield statute_url, elements

def generate_chapter_elements():
    for chapter_pdf_url in chapter_pdf_urls:
        elements = partition(url=chapter_pdf_url, content_type="application/pdf")
        yield chapter_pdf_url, elements

We prepare the sources for question generation by *chunking* them. We will do this again later using different chunking strategies when we store embedded chunks in a vector database for RAG.

In [ ]:
from langchain.docstore.document import Document as LangchainDocument
from langchain_text_splitters import RecursiveCharacterTextSplitter
from unstructured.documents.elements import Element


def chunk_elements_qa(elements: list[Element]) -> list[LangchainDocument]:
    # unstructured Element -> LangchainDocument for chunking
    docs = [
        LangchainDocument(
            element.text,
            metadata = {
                "url": element.metadata.url,
                "page_number": element.metadata.page_number,
            },
        )
        for element in elements
    ]
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=5000,
        chunk_overlap=750,
        separators=["\n\n", "\n", ".", " ", ""],
    )
    docs_processed = []
    for doc in docs:
        docs_processed += text_splitter.split_documents([doc])
    return docs_processed

#### 1.2 Setup question generation agents

It is necessary to call an LLM for each agent in the eval dataset generation and RAG processes, so let's write a function for them. We use `gpt-3.5-turbo` for the question generation agent.

In [ ]:
llm_client = openai.OpenAI()

def call_llm(
    client: openai.OpenAI,
    prompt: str,
    model: str = chat_models.GPT_3_5,
    temperature: int = 0.7,
    extra_messages: list | None = None,
):
    if extra_messages is None:
        extra_messages = []
    prompt_msg = {"role": "system", "content": prompt}
    response = client.chat.completions.create(
        model=model,
        messages=[prompt_msg, *extra_messages],
        max_tokens=1000,
        temperature=temperature,
    )
    return response.choices[0].message.content

Here is the prompt that is given to our question generation agent:

In [ ]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

For cost and time considerations, we generate a number of questions proportional to the length of the chunk.

In [ ]:
import random


def generate_qa_couples(
    chunks: list[LangchainDocument],
    max_num_questions: int,
    min_num_chars: int,
    chars_per_page: int,
    max_len_answer: int,
) -> list:
    char_count = sum([len(chunk.page_content) for chunk in chunks])
    if char_count < min_num_chars:
        return []

    question_count = min([max_num_questions, max([1, char_count // chars_per_page])])
    print(f"Generating {question_count} QA couples...")

    couples = []
    for sampled_context in tqdm(random.sample(chunks, question_count)):
        # Generate QA couple
        output_qa_couple = call_llm(
            llm_client,
            QA_generation_prompt.format(context=sampled_context.page_content),
        )
        try:
            question = output_qa_couple.split("Factoid question: ")[-1].split("Answer: ")[0].rstrip()
            answer = output_qa_couple.split("Answer: ")[-1].rstrip()
            assert len(answer) < max_len_answer, "Answer is too long"
            couples.append(
                {
                    "context": sampled_context.page_content,
                    "question": question,
                    "answer": answer,
                    "source_doc": sampled_context.metadata["url"],
                },
            )
        except:
            continue
    return couples

In [ ]:
def generate_n_qa_couples(
    n: int,
    max_questions_per_chunk: int = 4,
    min_chars_per_chunk: int = 500,
    chars_per_page: int = 2500,
    max_len_answer: int = 300,
):
    if n < max_questions_per_chunk:
        # so we generate exactly n couples
        max_questions_per_chunk = n
    tot_couples = []
    for src, elements in generate_chapter_elements():
        chunks = chunk_elements_qa(elements)
        couples = generate_qa_couples(
            chunks,
            # so we generate exactly n couples
            min([max_questions_per_chunk, n - len(tot_couples)]),
            min_chars_per_chunk,
            chars_per_page,
            max_len_answer,
        )
        tot_couples += couples
        if len(tot_couples) == n:
            break
    return tot_couples

In [ ]:
NUM_QUESTIONS = 3
couples = generate_n_qa_couples(NUM_QUESTIONS)
display(pd.DataFrame(couples).head(NUM_QUESTIONS))

#### 1.5 Setup question critique agents

The generated questions can be flawed in many ways.

>We use an agent to determine if a generated question meets the following criteria, given in [this paper](https://huggingface.co/papers/2312.10003):

- **Groundedness**: can the question be answered from the given context?
- **Relevance**: is the question relevant to users? For instance, *"What are some of Thomas Jefferson's beliefs regarding the rights and liberties of individuals?"* is not relevant for OpenProBono users.
- **Standalone**: is the question understandable free of any context, for someone with domain knowledge/Internet access? For instance, *"What does the term 'legal entity' refer to in this statute?"* is tailored for a particular statute, but unclear by itself.

>We systematically score functions with all these agents, and whenever the score is too low for any one of the agents, we eliminate the question from our eval dataset.
>
>💡 ***When asking the agents to output a score, we first ask them to produce its rationale. This will help us verify scores, but most importantly, asking it to first output rationale gives the model more tokens to think and elaborate an answer before summarizing it into a single score token.***"


In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to people learning about the legal system.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain specific legal definitions or entities like trier of fact or the Board of County Commissioners and still be a 5: it must simply be clear to an operator with access to legal documents what the question is about.

For instance, "Who decides the arbitration location without agreement?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [ ]:
def generate_qa_critiques(couples: list[dict]):
    print("Generating critique for each QA couple...")
    for output in tqdm(couples):
        evaluations = {
            "groundedness": call_llm(
                llm_client,
                question_groundedness_critique_prompt.format(context=output["context"], question=output["question"]),
            ),
            "relevance": call_llm(
                llm_client,
                question_relevance_critique_prompt.format(question=output["question"]),
            ),
            "standalone": call_llm(
                llm_client,
                question_standalone_critique_prompt.format(question=output["question"]),
            ),
        }
        try:
            for criterion, evaluation in evaluations.items():
                score, eval = (
                    int(evaluation.split("Total rating: ")[-1].strip()),
                    evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
                )
                output.update(
                    {
                        f"{criterion}_score": score,
                        f"{criterion}_eval": eval,
                    }
                )
        except:
            continue

>Now let us filter out bad questions based on our critique agent scores:

In [ ]:
def filter_questions(
        generated_questions: pd.DataFrame,
        min_groundedness: int,
        min_relevance: int,
        min_standalone: int,
):
    return generated_questions.loc[
        (generated_questions["groundedness_score"] >= min_groundedness)
        & (generated_questions["relevance_score"] >= min_relevance)
        & (generated_questions["standalone_score"] >= min_standalone)
    ]

In [ ]:
generate_qa_critiques(couples)
couples_df = pd.DataFrame.from_dict(couples)
couples_df = filter_questions(couples_df, 4, 4, 4)
display(couples_df.head(NUM_QUESTIONS))

>Now our synthetic evaluation dataset is complete! We can evaluate different RAG systems on this evaluation dataset.

Save the dataset to a file:

In [ ]:
with Path("./eval_dataset3.json").open("w") as f:
    f.write(couples_df.to_json())

### 2: Build our RAG System

We load the data for RAG into a `Collection` in our Milvus vector database.

To accomplish this, we must:

1. extract the content from our sources
2. chunk the extracted content
3. embed the chunks
4. insert them to the database

We already wrote generator functions to extract the content, `generate_statute_elements` and `generate_chapter_elements`, so our next step is chunking.

#### 2.1: Chunk sources into documents

>- In this part, **we split the documents from our knowledge base into smaller chunks**: these will be the snippets that are picked by the Retriever, to then be ingested by the Reader LLM as supporting elements for its answer.
>- The goal is to build semantically relevant snippets: not too small to be sufficient for supporting an answer, and not too large too avoid diluting individual ideas.
>
>Many options exist for text splitting:
>
>- split every n words / characters, but this has the risk of cutting in half paragraphs or even sentences
>- split after n words / character, but only on sentence boundaries
>- **recursive split** tries to preserve even more of the document structure, by processing it tree-like way, splitting first on the largest units (chapters) then recursively splitting on smaller units (paragraphs, sentences).
>
>To learn more about chunking, I recommend you read [this great notebook](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/5_Levels_Of_Text_Splitting.ipynb) by Greg Kamradt.
>
>[This space](https://huggingface.co/spaces/m-ric/chunk_visualizer) lets you visualize how different splitting options affect the chunks you get.

We need a function to chunk the content in a given source into documents. This function should accept a list of items containing extracted text and relevant metadata from the source. In our generator function, `partition` returns a list of `Element` objects. We pass these into a chunking function, `chunk_by_title`.

In [ ]:
from unstructured.chunking.title import chunk_by_title

chunking_function = chunk_by_title

#### 2.2: Embed documents

>The retriever acts like an internal search engine: given the user query, it returns the most relevant documents from your knowledge base.

We want to evaluate different embedding models. We represent them with a function that accepts text and returns vectors.

In order to easily test different models, our function also accepts an `EncoderParams` object that tells it which embedding model to use.

In [ ]:
embedding_function = encoders.embed_strs

#### 2.3: Create vector database

The document embeddings are stored in a vector database for fast vector searching during RAG.

Below is a function that creates a vector database we can use to store and retrieve the embedded documents. The `name` of the database will be determined by the chosen sources, chunking strategy, and embedding model evaluation parameters.

In [ ]:
create_collection_function = milvusdb.create_collection

#### 2.4: Insert documents

The last step is to insert the embedded documents into the newly created vector database.

An embedded document is represented by its corresponding `vectors`, `texts`, and `metadatas`. The vector database is given by a Milvus `Collection`.

In [ ]:
INSERT_BATCH_SIZE = 100

def insert_documents(vectors: list, texts: list, metadatas: list, collection: pymilvus.Collection):
    data = [vectors, texts, metadatas]
    num_chunks = len(vectors)
    pks = []
    for i in range(0, num_chunks, INSERT_BATCH_SIZE):
        batch_vector = data[0][i: i + INSERT_BATCH_SIZE]
        batch_text = data[1][i: i + INSERT_BATCH_SIZE]
        batch_metadata = data[2][i: i + INSERT_BATCH_SIZE]
        batch = [batch_vector, batch_text, batch_metadata]
        current_batch_size = len(batch[0])
        res = collection.insert(batch)
        pks += res.primary_keys
        if res.insert_count != current_batch_size:
            # the upload failed, try deleting any partially uploaded data
            bad_deletes = []
            for pk in pks:
                delete_res = collection.delete(expr=f"pk=={pk}")
                if delete_res.delete_count != 1:
                    bad_deletes.append(pk)
            bad_insert = (
                f"Failure: expected {current_batch_size} insertions but got "
                f"{res.insert_count}. "
            )
            if bad_deletes:
                bad_insert += f"Dangling data: {','.join(bad_deletes)}"
            else:
                bad_insert += "Any partially uploaded data has been deleted."
            return {"message": bad_insert}
    return {"message": "Success", "num_chunks": num_chunks}

#### 2.5 Populate the vector database

So far, the parameters we can control are:

1. Chunking strategy parameters
    - chunk hardmax: the maximum number of characters in a chunk
    - chunk softmax: the preferred maximum number of characters in a chunk (see `new_after_n_chars` in `chunk_by_title` for more)
    - overlap: the number of characters to overlap between consecutive chunks
2. Embedding model

There are also some parameters we can't currently control:

1. Document loader (unstructured)
2. Question generation chunking strategy (LangChain's `RecursiveCharacterTextSplitter`)
2. Base RAG chunking strategy (unstructured's `chunk_by_title`)
3. Vector Index (Zilliz autoindex)

Once we decide on values for the parameters we can control, we can chunk and embed our sources into a vector database. We can then use the database to generate a synthetic dataset of question-answer pairs and/or evaluate agents' responses to a generated (or loaded) dataset of questions.

In [ ]:
def populate_database(collection: pymilvus.Collection, chunk_hardmax: int, chunk_softmax: int, overlap: int, encoder: encoders.EncoderParams):
    for src, elements in generate_statute_elements():
        chunks = chunking_function(
            elements,
            max_characters=chunk_hardmax,
            new_after_n_chars=chunk_softmax,
            overlap=overlap,
        )
        num_chunks = len(chunks)
        texts, metadatas = [], []
        for i in range(num_chunks):
            texts.append(chunks[i].text)
            metadatas.append(chunks[i].metadata.to_dict())
        vectors = embedding_function(texts, encoder)
        result = insert_documents(vectors, texts, metadatas, collection)
        if result["message"] != "Success":
            break

We may want to rerun this notebook with data that has already been inserted to Milvus. If so, we need different generator functions to load the data from Milvus rather than the source URLs themselves.

In [ ]:
from unstructured.documents.elements import ElementMetadata, Text


def vdb_source_documents(collection_name: str, source: str):
    expr = f"metadata['url']=='{source}'"
    hits = milvusdb.get_expr(collection_name, expr)["result"]
    for i in range(len(hits)):
        hits[i]["url"] = hits[i]["metadata"]["url"]
        hits[i]["page_number"] = hits[i]["metadata"]["page_number"]
        del hits[i]["pk"]
        del hits[i]["metadata"]
    return [
        Text(
            text=hit["text"],
            metadata=ElementMetadata(
                url=source,
                page_number=hit["page_number"],
            ),
        ) for hit in hits
    ]

def load_statute_elements(collection_name: str):
    for chapter in chapter_names:
        statute_urls = load_statute_urls(chapter)
        for statute_url in statute_urls:
            yield vdb_source_documents(collection_name, statute_url)

def load_chapter_elements(collection_name: str):
    for chapter_pdf_url in chapter_pdf_urls:
        yield vdb_source_documents(collection_name, chapter_pdf_url)

#### 2.6 Reader - LLM 💬

>In this part, the LLM Reader reads the retrieved documents to formulate its answer.

OpenProBono offers various LLM Readers using models offered by OpenAI, HuggingFace, and more. We will create a simple LLM Reader connected to the OpenAI API for this notebook.

In [ ]:
RAG_PROMPT_TEMPLATE = """
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.

Context:
{context}

Question: {question}
"""

In [ ]:
def answer_with_rag(
    collection_name: str,
    question: str,
    k: int = 7,
) -> tuple[str, list[dict]]:
    """Answer a question using RAG."""
    # Gather documents with retriever
    relevant_docs = milvusdb.query(collection_name, question, k)
    relevant_docs = [doc.entity.get("text") for doc in relevant_docs["result"]]  # keep only the text

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {i!s}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    answer = call_llm(llm_client, final_prompt, temperature=0)

    return answer, relevant_docs

### 3: Benchmarking the RAG System

>The RAG system and the evaluation datasets are now ready. The last step is to judge the RAG system's output on this evlauation dataset.
>
>To this end, **we setup a judge agent**. ⚖️🤖
>
>Out of the different RAG evaluation metrics, we choose to focus only on faithfulness since it the best end-to-end metric of our system's performance.

>💡 *In the evaluation prompt, we give a detailed description each metric on the scale 1-5, as is done in Prometheus's prompt template: this helps the model ground its metric precisely. If instead you give the judge LLM a vague scale to work with, the outputs will not be consistent enough between different examples.*
>
>💡 *Again, prompting the LLM to output rationale before giving its final score gives it more tokens to help it formalize and elaborate a judgement.*

In [ ]:
def run_rag_tests(
    eval_dataset: pd.DataFrame,
    collection_name: str,
    output_file: str,
    verbose: bool | None = True,
    test_settings: str = "",  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with Path(output_file).open() as f:
            outputs = json.load(f)
    except:
        outputs = []

    for _, example in tqdm(eval_dataset.iterrows(), total=len(eval_dataset)):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(collection_name, question)
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": list(relevant_docs),
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with Path(output_file).open("w") as f:
            json.dump(outputs, f)

In [ ]:
EVALUATION_SYSTEM_MSG = "You are a fair evaluator language model."

EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

In [ ]:
evaluator_name = chat_models.GPT_3_5
eval_temperature = 0


def evaluate_answers(
    answer_path: str,
    evaluator_name: str,
) -> None:
    """Evaluate generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if Path(answer_path).is_file():  # load previous generations if they exist
        with Path(answer_path).open() as f:
            answers = json.load(f)

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = EVALUATION_PROMPT.format(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        eval_msg = {"role": "user", "content": eval_prompt}
        eval_result = call_llm(llm_client, EVALUATION_SYSTEM_MSG, evaluator_name, eval_temperature, [eval_msg])
        feedback, score = (item.strip() for item in eval_result.split("[RESULT]"))
        experiment[f"eval_score_{evaluator_name}"] = score
        experiment[f"eval_feedback_{evaluator_name}"] = feedback

        with Path(answer_path).open(mode="w") as f:
            json.dump(answers, f)

>🚀 Let's run the tests and evaluate answers!👇

In [ ]:
from datetime import UTC, datetime

if not Path("./output").exists():
    Path.mkdir("./output")

vdb_basename = "Evaluation_" + datetime.now(UTC).strftime("%Y%m%d")
idx = 1
for chunk_hardmax, chunk_softmax, overlap in [(2000, 1000, 300), (5000, 2000, 750)]:
    for encoder in [encoders.DEFAULT_ENCODER, encoders.EncoderParams(encoders.OPENAI_3_LARGE, 1536)]:
        settings_name = (
            f"chunk-hardmax:{chunk_hardmax}"
            f"_chunk-softmax:{chunk_softmax}"
            f"_embeddings:{encoder.name}"
            f"_dim:{encoder.dim}"
            f"_reader-model:{chat_models.GPT_3_5}"
        )
        output_file_name = f"./output/rag_{settings_name}.json"

        print(f"Running evaluation for {settings_name}:")

        print("Loading knowledge base embeddings...")
        collection_name = f"{vdb_basename}_{idx}"
        idx += 1
        description = (
            f"Hardmax = {chunk_hardmax}, "
            f"Softmax = {chunk_softmax}, "
            f"Overlap = {overlap}, "
            f"Encoder = {encoder.name}."
        )
        if pymilvus.utility.has_collection(collection_name):
            collection = pymilvus.Collection(collection_name)
        else:
            collection = create_collection_function(
                collection_name,
                description,
                encoder=encoder,
            )
            populate_database(collection, chunk_hardmax, chunk_softmax, overlap, encoder)

        print("Running RAG...")
        run_rag_tests(
            eval_dataset=couples_df,
            collection_name=collection_name,
            output_file=output_file_name,
            verbose=True,
            test_settings=settings_name,
        )

        print("Running evaluation...")
        evaluate_answers(
            output_file_name,
            evaluator_name,
        )